In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install imbalanced-learn
!pip install delayed

     |████████████████████████████████| 226 kB 11.7 MB/s 
     |████████████████████████████████| 85 kB 4.0 MB/s 


In [2]:
#!pip install imbalanced-learn
#!pip install delayed

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import sklearn
from imblearn.over_sampling import RandomOverSampler

In [7]:
# path of dataset
data_path = "./drive/MyDrive/Colab Notebooks/data/Action Detection Dataset/Training Dataset/"
data_arrays_path = "./drive/MyDrive/Colab Notebooks/data_arrays/Action Detection Dataset/Training Dataset/"

# assigning parameters for MFCC feature extraction
max_len_nfft = 649#changed
n_mfcc = 128

In [8]:
# Saving data to array file
save_data_to_array(data_path=data_path, data_arrays_path=data_arrays_path, max_len=max_len_nfft, n_mfcc=n_mfcc)

Saving vectors of label - 'chalao': 100%|██████████| 1129/1129 [00:41<00:00, 26.95it/s]
Saving vectors of label - 'RealActionDetection Negative': 100%|██████████| 1089/1089 [00:28<00:00, 38.60it/s]


Streaming output truncated to the last 5000 lines.
203
203
203
203
203
203
203
203
204
204
204
204
204
204
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
205
206
206
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
207
208
208
208
208
208
208
208
208
208
208
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
209
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
210
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
211
2

In [9]:
# Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test(data_path=data_path, data_arrays_path=data_arrays_path, split_ratio=0.8)
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (5593, 128, 649)
X_test:  (1399, 128, 649)


In [10]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')

# reshaping the data for oversampling
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])

# fit and apply the transform
X_train, y_train = oversample.fit_resample(X_train, y_train)
X_test, y_test = oversample.fit_resample(X_test, y_test)

# converting it back to original shape
X_train = X_train.reshape(X_train.shape[0], n_mfcc, max_len_nfft)
X_test = X_test.reshape(X_test.shape[0], n_mfcc, max_len_nfft)

print("X_train: ", X_train.shape, end='\t')
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape, end='\t')
print("y_test: ", y_test.shape)

X_train:  (6441, 128, 649)	y_train:  (6441,)
X_test:  (1626, 128, 649)	y_test:  (1626,)


In [11]:
# Feature dimension
epochs = 100
batch_size = 32
num_classes = 11

# model
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, input_shape=(n_mfcc, max_len_nfft, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=256, kernel_size=2, activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(GlobalAveragePooling2D())

model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 127, 648, 16)      80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 324, 16)      0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 63, 324, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 323, 32)       2080      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 31, 161, 32)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 31, 161, 32)       0

In [12]:
model.compile(loss="SparseCategoricalCrossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [13]:
# model.fit(X_train, y_train_hot, epochs=epochs, validation_data=(X_test, y_test_hot))
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), shuffle=True)

Epoch 1/100
202/202 [==============================] - 31s 102ms/step - loss: 2.1635 - accuracy: 0.2105 - val_loss: 2.2959 - val_accuracy: 0.1365
Epoch 2/100
202/202 [==============================] - 19s 94ms/step - loss: 1.9138 - accuracy: 0.2849 - val_loss: 2.1626 - val_accuracy: 0.2251
Epoch 3/100
202/202 [==============================] - 19s 95ms/step - loss: 1.8347 - accuracy: 0.3232 - val_loss: 2.1616 - val_accuracy: 0.2282
Epoch 4/100
202/202 [==============================] - 20s 99ms/step - loss: 1.7385 - accuracy: 0.3611 - val_loss: 2.0268 - val_accuracy: 0.2731
Epoch 5/100
202/202 [==============================] - 19s 94ms/step - loss: 1.6362 - accuracy: 0.3964 - val_loss: 1.9229 - val_accuracy: 0.3118
Epoch 6/100
202/202 [==============================] - 19s 95ms/step - loss: 1.5576 - accuracy: 0.4294 - val_loss: 1.8708 - val_accuracy: 0.3629
Epoch 7/100
202/202 [==============================] - 19s 94ms/step - loss: 1.4024 - accuracy: 0.4869 - val_loss: 1.5959 - val_a

In [14]:
model.save("./Models/Action Detection Model")

INFO:tensorflow:Assets written to: ./Models/Action Detection Model/assets


In [15]:
model.save("./drive/MyDrive/Colab Notebooks/Models/Action Detection Model")

INFO:tensorflow:Assets written to: ./drive/MyDrive/Colab Notebooks/Models/Action Detection Model/assets


In [ ]:
model=keras.models.load_model("./Models/Action Detection Model")

In [ ]:
test_data = wav2mfcc("/content/Ruha kitchen ki light jalao.wav", [], 128, 603)
test_data = test_data.reshape(1, 128, 603)
result = model.predict(test_data)
print(result)
result = result.argmax(axis=1)
print(result)

[[7.2996593e-03 7.9618934e-07 1.8532288e-06 5.4525766e-03 6.3499908e-07
  1.0816368e-06 5.2376916e-05 3.3508838e-04 8.7956745e-05 9.8676795e-01]]
[9]


0 -Previous ,[1] unmute , [2] slow, [3] Next , [4] mute ,5 Fast ,6 off karo ,7 band karo ,8 on karo ,9 chalao 